In [1]:
%pip install openai pydantic jinja2 python_dotenv ipywidgets

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import tomllib

from jinja2 import Template
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv

from typing import Literal, Union

load_dotenv()

with open("data/example.toml", "rb") as f:
    config = tomllib.load(f)

version = config["version"]
system = config["system"]
rounds = config["rounds"]

print(f"Version: {version}, Rounds: {len(rounds)}")

message_pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
class Message(BaseModel):
    role: Literal["system", "assistant", "user"]
    content: str   

    def dict(self) -> dict:
        current_key = None
        parsed = {}
        lines = self.content.splitlines()
        # parse the content line by line
        for line in lines:
            res = message_pattern.search(line)
            if res is not None:
                current_key = res.group(1)
                if res.group(2) is not None:
                    parsed[current_key] = res.group(2)
                else:
                    parsed[current_key] = ""
            elif current_key is not None:
                parsed[current_key] += "\n" + line
        return parsed

class Round(BaseModel):
    id: str
    tags: list[str]
    next: list[str] | None
    max_turns: int
    rules: list[str]
    initial: str
    
    def __init__(self, **kargs):
        super().__init__(**kargs)
    
    def get_messages(self, system) -> list[Message]:
        system_content = Template(system).render({"rules": self.rules})
        return [
            Message(role="system", content=system_content),
            Message(role="assistant", content=self.initial)
        ]

def chat(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)
        
def getRoundById(id:str) -> Round:
    for r in rounds:
        if r["id"] == id:
            return Round(**r) 
    return None

current_round = getRoundById("r1")

Version: 0.0.2, Rounds: 1


In [10]:
messages = current_round.get_messages(system)
print(messages[-1].dict())

{'narration': '\nYou have been investigating a series of murders in the town for many years.\nThe recent clues led you to a abandoned house in the forest. Now you are standing in front of the house.', 'choices': '\n1. Enter the house from the front door.\n2. Look around the house to find some clues.\n3. Drink the alcohol to gain some courage.'}
